# Vector Databases

Em **NLP** (*Natural Language Processing*), frases e documentos são frequentemente representados como **vetores numéricos**. Isto ocorre, por exemplo, para fornecer integração com *Machine Learning* (modelos de **NLP** frequentemente produzem ou utilizam vetores em suas operações) e pesquisa eficiente por similaridade (ao buscar documentos ou frases semelhantes).

Um banco de dados vetorial (*Vector Database*) ou mecanismo de busca vetorial é um **banco de dados** que pode armazenar tais **vetores** (listas de números de comprimento fixo) juntamente com outros itens de dados, enquanto permite recuperar informação de forma **eficiente** e **escalável**.

Nesta aula, exploraremos desde a representação vetorial de textos até uma aplicação de **RAG** (*Retrieval-Augmented Generation*) com **LLM** (*Large Language Model*).

## Criar ambiente virtual

É extremamente recomendável que você realize esta aula em um novo ambiente virtual.

- Criar venv com `conda`

```console
conda create -n mdvector python=3.10
conda activate mdvector
```

- Criar venv com `python -m`

```console
python -m venv venv

# Ativar no Windows
venv\Scripts\activate

# Ativar no Linux/MacOS
source venv/bin/activate
```

## Instalação das Libs

Após criar e ativar seu ambiente virtual, realize a instalação das dependências:

**Atenção:** se for realizar a instalação pelo notebook, garanta que o notebook está executando com o ambiente correto!

In [23]:
# !python -m pip install -r requirements.txt

## Chave OpenAI

Nesta aula, iremos utilizar a API da OpenAI pela Azure.

Utilize as credenciais fornecidas pelo professor. Crie um arquivo `.env` a partir do arquivo `.env.example` e faça a configuração das variáveis de ambiente.

**ATENÇÃO**: Utilize esta chave com cuidado. Não desperdice recursos, não execute células múltiplas vezes de forma desnecessária e, mais importante, **NÃO VAZE A CHAVE** publicamente nem compartilhe com terceiros!

In [24]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Textos: Representação Vetorial

Vetores são representações matemáticas de dados em um espaço de alta dimensão. Nesse espaço, cada dimensão corresponde a uma característica dos dados, com o número de dimensões variando de algumas centenas a dezenas de milhares, dependendo da complexidade dos dados representados.

A maneira mais simples de vetorizar textos é utilizando o método **Bag of Words** (**BoW**). Nele, cada texto é representado pela ocorrência (ou frequência) de suas palavras. Ele ignora a ordem das palavras e considera apenas a presença ou ausência delas no documento.

Por exemplo, vamos supor um dicionário de uma língua com apenas três palavras:

```python
dic = ["oi", "bom", "horrível"]
```

Agora, considere as seguintes frases:

```python
frases = [
    "oi tudo bom?",
    "bom dia",
    "foi horrível",
]

Podemos fazer uma representação vetorial das frases pelos seguintes vetores:

```python
vetores = [
    [1, 1, 0], # Tem palavra "oi", tem "bom", não tem "horrível"
    [0, 1, 0], # Não tem "oi", tem "bom", não tem "horrível"
    [0, 0, 1], # Não tem "oi", não tem "bom", tem "horrível"
]
```

Perceba que ignoramos as palavras não pertencentes ao dicionário.

Vamos representar isto em código-fonte!

In [25]:
import plotly.graph_objects as go
import numpy as np

# Frases personalizadas
frases = [
    "oi tudo bom?",
    "bom dia",
    "foi horrível",
    "oi, tive um dia bom e horrível ao mesmo tempo"
]

# Vetores das frases, considerando o dicionário ["oi", "bom", "horrível"]
vetores = np.array([
    [1, 1, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 1],
])

E visualizar um gráfico dos vetores:

In [26]:

# Cores diferentes para cada ponto
cores = ["#636EFA", "#EF553B", "#00CC96", "#666666"]

# Criar uma figura do Plotly
fig = go.Figure()

# Adicionar pontos ao gráfico
for vector, color, frase in zip(vetores, cores, frases):
    fig.add_trace(go.Scatter3d(
        x=[vector[0]],
        y=[vector[1]],
        z=[vector[2]],
        mode="markers",
        marker=dict(size=5, color=color),
        text=f"{frase} {vector}",
        hoverinfo="text"
    ))

# Definir título e rótulos dos eixos com limites
fig.update_layout(
    title="Vetores de Frases - Gráfico de Pontos 3D",
    scene=dict(
        xaxis=dict(title="Eixo oi", range=[0, 2]),
        yaxis=dict(title="Eixo bom", range=[0, 2]),
        zaxis=dict(title="Eixo horrível", range=[0, 2])
    ),
    showlegend=False
)

# Mostrar o gráfico
fig.show()


## Similaridade entre vetores

Para analisar quão similares são dois textos, uma alternativa é calcular a similaridade do cosseno entre os dois vetores que representam os textos.

A similaridade do cosseno mede o cosseno do ângulo entre dois vetores. É uma medida popular em processamento de linguagem natural e recuperação de informações e pode ser calculada como:


$$ s(A,B)= \frac{\sum(A_i \times B_i)}{\sqrt{\sum(A_i^2)} ⋅ \sqrt{\sum(B_i^2)}} = \frac{A⋅B}{∥A∥∥B∥}$$

Veja uma definição em python:

In [27]:
import numpy as np

def cosine_similarity(vector1, vector2):
    cosine_sim = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    return cosine_sim

**Dica**: Também é possível utilizar a função da biblioteca `scipy`:

```python
from scipy.spatial.distance import cosine
```

Vamos calcular a similaridade entre as frases:

In [28]:
# Calcular a matriz de distância cosseno
num_frases = len(frases)
matriz_distancia = np.zeros((num_frases, num_frases))

for i in range(num_frases):
    for j in range(num_frases):
        matriz_distancia[i, j] = cosine_similarity(vetores[i], vetores[j])

E visualizar o mapa de calor:

In [29]:
# Criar uma figura do Plotly
fig = go.Figure(data=go.Heatmap(
    z=matriz_distancia,
    x=frases,
    y=frases,
    colorscale="YlOrRd"
))

# Definir título e rótulos dos eixos
fig.update_layout(
    title="Matriz de Distância Cosseno",
    xaxis_title="Frases",
    yaxis_title="Frases"
)

# Mostrar o gráfico
fig.show()

**Exercício**: A frase `"bom dia"` é mais similar à frase `"oi tudo bom?"` ou à frase `"foi horrível"`?

In [30]:
# Sua resposta aqui!
"oi tudo bom?"

'oi tudo bom?'

**Exercício**: Qual seria a representação vetorial da frase `"olá tenham todos um dia horrível"` considerando o mesmo dicionário do exemplo?

<a href="#" title="[0, 0, 1] pois apenas a palavra 'horrivel' está no dicionário e no texto!">Passe o mouse aqui para conferir resposta.</a>

In [31]:
# Sua resposta aqui!
[0, 0, 1]

[0, 0, 1]

Outras alternativas incluem o uso de:

- distância euclidiana:

$$ d(A, B) = \sqrt{\sum_{i=1}^{n} (A_i - B_i)^2} $$

- distância euclidiana ao quadrado ou norma L2 ao quadrado (equivalentes):

$$d(\mathbf{A}, \mathbf{B})^2 = \|\mathbf{A} - \mathbf{B}\|_2^2 = \sum_{i=1}^{n} (A_i - B_i)^2$$

- produto interno:

$$\langle \mathbf{A}, \mathbf{B} \rangle = 1 - \sum_{i=1}^{n} A_i B_i$$

**Atenção**: neste bloco, definimos as alternativas como funções de distância. Em busca por similaridade, a similaridade é maximizada, enquanto que a distância é minimizada (onde você normalmente procuraria alta similaridade, você vai querer baixa distância).

## Indo além do BoW: Word Embeddings

Embora seja simples e eficiente, o **BoW** tem várias limitações. Primeiramente, ele não captura a semântica das palavras. Por exemplo, as palavras `"cão"` e `"cachorro"` são sinônimos, mas no **BoW**, elas são tratadas como palavras completamente diferentes.

Além disso, **BoW** também não considera a **ordem das palavras**, o que pode ser problemático para entender o contexto. Por exemplo, as frases `"o gato mordeu o cachorro"` e `"o cachorro mordeu o gato"` têm significados muito diferentes, mas no **BoW**, elas podem ser representadas da mesma forma se contiverem as mesmas palavras.

Esses problemas levam à necessidade de representações de texto mais avançadas, que capturam a semântica e o contexto das palavras. É aqui que os *embeddings* entram em cena. *Embeddings* são representações vetoriais densas onde **palavras com significados semelhantes têm representações semelhantes**.

<img src="img/embedding.png">


Diferentemente do **BoW**, que representa cada palavra como um valor único em um vetor esparso, os *embeddings* mapeiam palavras para um espaço **vetorial contínuo** de relativa baixa dimensão. Isso permite que relações semânticas e contextuais sejam preservadas.

Por exemplo, em um espaço de *embeddings*, as palavras `"rei"` e `"rainha"` estarão próximas uma da outra, refletindo sua semelhança semântica, enquanto estarão longe de uma palavra como `"carro"`.

<style>
    .image-container {
        background-color: white;
        display: inline-block;
    }
</style>

<div class="image-container">
    <img src="img/emb_vectors.png">
</div>

Os modelos utilizados para *embedding* são geralmente treinados em grandes corpora de texto. Esses modelos aprendem as representações vetoriais das palavras com base em seu contexto de uso nas frases, capturando assim **nuances semânticas** e **sintáticas** que **BoW** não consegue.

**Dica**: os modelos modernos de redes neurais em **NLP** processam **tokens** em vez de palavras. Um token é a menor unidade de texto que pode ser processada pelo modelo. Tokens podem ser palavras, caracteres, sinais de pontuação, símbolos ou partes de palavras.

Acesse o link https://platform.openai.com/tokenizer e experimente o tokenizador online da OpenAI. Digite alguns textos, com pontuações, e observe os tokens gerados.

<img src="img/openai_tokens.png">

### SentenceTransformers Embeddings

Vamos utilizar a biblioteca [**SentenceTransformers**](https://sbert.net/) para obter o embedding de textos.

Veja mais em https://sbert.net/

Podemos especificar o modelo com (veja mais modelos em https://www.sbert.net/docs/sentence_transformer/pretrained_models.html):

In [32]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "multi-qa-MiniLM-L6-cos-v1"
embedding_model = SentenceTransformer(embedding_model_name)

E utilizar para fazer o embedding de uma frase:

In [33]:
vetor = embedding_model.encode("meu cachorro é muito fofo")

Conferindo o resultado:

In [34]:
print(f"O vetor tem {len(vetor)} dimensões")

O vetor tem 384 dimensões


In [35]:
vetor

array([-3.51005569e-02, -8.63294210e-03,  1.45162363e-02, -1.08098576e-03,
       -1.89534370e-02, -1.32326018e-02,  1.51183426e-01,  3.72115001e-02,
       -4.36554337e-03,  1.29401712e-02, -2.70885397e-02, -1.61787942e-01,
       -1.72241416e-03,  2.98229679e-02, -6.25767186e-02,  1.56986807e-02,
        8.10203850e-02, -4.34880927e-02, -9.78889409e-03, -1.77063420e-02,
        3.41576412e-02,  4.09843726e-03, -5.16073331e-02,  3.36873941e-02,
       -7.39732012e-02,  2.85558272e-02, -5.30630024e-03,  4.62772027e-02,
       -6.50665984e-02, -1.20001979e-01,  7.91811850e-03,  4.42371964e-02,
        9.64584500e-02, -1.18117128e-02,  1.90767497e-02,  1.69214401e-02,
        6.86635682e-03, -3.02379057e-02,  4.60491925e-02,  5.05991373e-03,
       -8.05164948e-02, -3.18694524e-02,  5.23832627e-02,  5.35883605e-02,
        3.08352932e-02, -4.22245599e-02,  3.44336359e-03,  1.72899943e-02,
        8.47037509e-02,  2.75640320e-02, -7.48745799e-02,  2.59381533e-03,
       -3.50019820e-02,  

Agora, vamos definir algumas frases:

In [36]:
frases_novas = [
    "meu cachorro é muito fofo",
    "iphone 15 pro max é muito caro",
    "quero um pet que é amigo das crianças",
    "smartphones estão focando em IA",
    "apple encerra programa de carro autônomo",
]

E fazer o embedding de cada frase:

In [37]:
vetores_novos = embedding_model.encode(frases_novas)

Obtemos como resultado:

In [38]:
print(f"A matriz possui {len(vetores_novos)} vetores")
print(f"Cada vetor possui {len(vetores_novos[0])} dimensões")
print(f"Ou seja, temos uma matriz {len(vetores_novos)}x{len(vetores_novos[0])}")

A matriz possui 5 vetores
Cada vetor possui 384 dimensões
Ou seja, temos uma matriz 5x384


Vamos calcular a similaridade entre as frases pelo cálculo da similaridade cosseno entre os seus vetores:

Confira quais frases são mais similares entre si.

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import plotly.express as px

similarity_matrix = cosine_similarity(vetores_novos)

fig = px.imshow(similarity_matrix, 
                x=frases_novas, 
                y=frases_novas, 
                color_continuous_scale="Viridis",
                labels={"x": "Frases", "y": "Frases", "color": "Similaridade"})
fig.update_layout(title="Heatmap de Similaridades Cosseno")
fig.show()

## Busca

Vamos supor que possuímos muitos textos representados de forma vetorial. Os textos poderiam ser parágrafos de diversos livros ou notícias. Se o usuário tiver alguma questão e desejar encontrar os vetores mais semelhantes à sua pergunta. Por exemplo:

```python
"Quais são os objetivos da área de tecnologia em 2024?"
```

Como isto poderia ser feito?

**Exercício**: Como você faria para realizar esta busca? Quais seriam os passos?

<a href="#" title="1) fazer embedding do texto da pergunta 2) calcular a distância entre o vetor do texto da pergunta e todos os vetores armazenados 3) retornar os mais semelhantes.">Pare o mouse aqui para conferir resposta.</a>

In [40]:
# Sua resposta AQUI

'''
1) Faria o embedding de todas as frases
2) Calcularia a distancia cosseno
3) retornaria as mais similares
'''

'\n1) Faria o embedding de todas as frases\n2) Calcularia a distancia cosseno\n3) retornaria as mais similares\n'

**Exercício**: Comparar o vetor da pergunta com todos os vetores é eficiente de um ponto de vista computacional?

<a href="#" title="Não! Pois seria O(kn), onde k é o tamanho do vetor e n é a quantidade de vetores. Apesar de ser linear no número de vetores, seria ineficiente ao considerar um número elevado de vetores.">Pare o mouse aqui para conferir resposta.</a>

In [41]:
# Sua resposta AQUI

'''
Nao, eh bastante ineficiente, resultaria num algoritmo O(kn)
'''

'\nNao, eh bastante ineficiente, resultaria num algoritmo O(kn)\n'

**Exercício**: Utilizar um SGBD relacional poderia ajudar? Pense em como um SGBD realiza buscas e qual seria o impacto nesta situação

In [42]:
# Sua resposta AQUI
'''
Relacional nao, pois os dados nao podem ser indexados da mesma forma que sao em SGBDs relacionais
'''

'\nRelacional nao, pois os dados nao podem ser indexados da mesma forma que sao em SGBDs relacionais\n'

## ChromaDB

Da mesma maneira que **SGBDs** como o **MySQL** implementam uma série de recursos para trabalhar com dados segundo o modelo relacional, também temos opções para bancos de dados vetoriais.

Nesta aula, iremos utilizar o [**Chroma DB**](https://www.trychroma.com/), mas temos outras opções comumente utilizadas:

- Pinecone (cloud)
- Milvus
- Qdrant
- PostgreSQL (um SGBD com módulo pgVector para vector DB)

<img src="img/chroma.svg">

Como exemplo, vamos utilizar o livro da Alice no País das Maravilhas (`data/alice.txt`).

Vamos dividir o livro em pedaços ou *chunks*:

In [43]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

chunk_size = 300

raw_documents = TextLoader("data/alice.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 321, which is longer than the specified 300
Created a chunk of size 702, which is longer than the specified 300
Created a chunk of size 729, which is longer than the specified 300
Created a chunk of size 424, which is longer than the specified 300
Created a chunk of size 726, which is longer than the specified 300
Created a chunk of size 520, which is longer than the specified 300
Created a chunk of size 427, which is longer than the specified 300
Created a chunk of size 626, which is longer than the specified 300
Created a chunk of size 573, which is longer than the specified 300
Created a chunk of size 478, which is longer than the specified 300
Created a chunk of size 664, which is longer than the specified 300
Created a chunk of size 304, which is longer than the specified 300
Created a chunk of size 597, which is longer than the specified 300
Created a chunk of size 429, which is longer than the specified 300
Created a chunk of size 476, which is longer tha

Veja um documento gerado:

In [44]:
documents[0]

Document(metadata={'source': 'data/alice.txt'}, page_content='capa\neBookLibris\nLewis Carroll\nALICE NO PAÍS DAS MARAVILHAS\nTradução de Clélia Regina Ramos\nEditorial Arara Azul\n \nAlice no País das Maravilhas\nLewis Carroll\n(Charles Lutwidge Dodgson 1832-1898)\nIlustrações\nSir John Tenniel (1820-1914)\nTradução\nClélia Regina Ramos\nmamalag@uol.com.br')

Então, criamos o banco de dados. Observe que iremos aplicar o `embedding_model`, que irá requerer chamadas à SentenceTransformers. Ainda, será criada uma pasta `alice_chroma_*`.

Mas antes, criaremos uma classe para possibilitar que o LangChain consiga trabalhar com a SentenceTransformers:

In [ ]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return self.model.encode(documents)

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0]

Finalmente, criamos nossa *VectorStore* com os embeddings e os documentos.

In [ ]:
from langchain_chroma import Chroma

chroma_embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)
data_path = f"./alice_chroma_{embedding_model_name.lower()}"

# Alterando função: `l2` (norma l2 ao quadrado), `cosine` (similaridade cosseno), `ip` (inner product: produto interno)
# https://docs.trychroma.com/docs/collections/configure
collection_metadata = {"hnsw:space": "cosine"}

db = Chroma.from_documents(
    documents,
    chroma_embedding_model,
    persist_directory=data_path,
    collection_metadata=collection_metadata,
)

Após executar uma vez, comente a célula acima que cria o `db` (uma vez que ele já existe). Então, vamos passar a abrir direto do arquivo (VectorStore)!

In [ ]:
from langchain_chroma import Chroma

db = Chroma(
    embedding_function=chroma_embedding_model, persist_directory=data_path
)

Agora, podemos utilizar o `db` para recuperar textos cujos vetores sejam mais similares à determinada pergunta:

In [ ]:
pergunta = "O que a lagarta tirou da boca?"

Para isto, será necessário calcular o vetor da pergunta (utilizando o mesmo modelo de *embedding* aplicado aos dados).

In [ ]:
embedding_pergunta = chroma_embedding_model.embed_query(pergunta)

print(f"O vetor da pergunta tem dimensão {len(embedding_pergunta)}.")

Então, podemos encontrar os `k` vetores mais similares:

In [ ]:
docs_resposta = db.similarity_search_by_vector(embedding_pergunta, k=5)

for doc in docs_resposta:
    # Exibir até 150 primeiros caracteres do conteúdo
    print(f"{doc.page_content[:150]}...")
    print("-"*50)

**Exercício**: teste com outras perguntas! Confira no arquivo `data/alice.txt` se as respostas fazem sentido.

In [ ]:
# Sua resposta AQUI!

**Exercício**: Como o Chroma encontra os vetores mais similares sem precisar comparar o vetor da pergunta com todos os vetores presentes na VectorStore? Procure por *Hierarchical Navigable Small Worlds (HNSW)* e explique como o *HNSW* indexa vetores e realiza busca por similaridade.

In [ ]:
# Sua resposta AQUI!

Por enquanto apenas retornamos os textos cujos vetores são mais semelhantes à pergunta. O texto é um recorte do texto original e o usuário precisa ler os textos e procurar a resposta.

## RAG

**RAG** (**Retrieval Augmented Generation**), é uma técnica que combina **recuperação de informações** com um modelo **LLM** de geração de texto para melhorar os resultados. Este método fornece ao modelo informações que podem estar mais atualizadas do que o conjunto de dados usado no treinamento.

Ainda, pode prover informações específicas de um contexto, por exemplo, quando queremos respostas envolvendo apenas os dados de um PDF empresarial privado.

A estrutura de uma solução RAG envolve três etapas principais:

1) Identificação de documentos relevantes que refletem o contexto da pergunta
2) Combinação desse contexto com um prompt que contém instruções específicas
3) geração do texto utilizando um LLM.

Nosso prompt será construído com:

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
Você é um assistente para tarefas de resposta a perguntas. Use as seguintes partes do contexto recuperado para responder à pergunta. Se você não sabe a resposta, basta dizer que não sabe. Use no máximo três frases e mantenha a resposta concisa.

Pergunta: {question} 

Contexto: {context} 

Resposta:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"]
)

Vamos especificar onde os textos de contexto são recuperados. Iremos utilizar o `db` (Chroma DB construído anteriormente).

In [ ]:
retriever = db.as_retriever()

Como **LLM**, iremos utilizar o modelo `gpt-4.1-nano`.

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o_MacielVidal_Chave1",
    api_version="2025-01-01-preview",
    temperature=0,
    max_tokens=1000,
    timeout=30,
    max_retries=2,
    model="gpt-4.1-nano",
)

E criar uma *RAG chain* com:

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)


Então, podemos fazer nossas perguntas, que serão respondidas utilizando o texto da Alice como referência.

In [ ]:
rag_chain.invoke("Qual o título do livro? E do terceiro capítulo?")

In [ ]:
rag_chain.invoke("Quem é o coelho?")

In [ ]:
rag_chain.invoke("O que a lagarta tirou da boca? Com quem a lagarta estava? O que ela disse?")

Para verificar que de fato o contexto é importante, vamos criar um *retriever* vazio e tentar repetir a pergunta. Perceba que, como não temos os dados do livro da Alice, a API do ChatGPT não irá responder corretamente (faltou contexto).

In [ ]:
empty_retriever = Chroma(embedding_function=chroma_embedding_model).as_retriever()

rag_chain_no_retriever = (
    {
        "context": empty_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_no_retriever.invoke("O que a lagarta tirou da boca? Com quem a lagarta estava? O que ela disse?")

**Dica:** Os prompts podem ser obtidos direto do *hub* langchain:

```python
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
```

Veja mais em https://smith.langchain.com/hub/rlm/rag-prompt e https://smith.langchain.com/hub/rlm

## Chainlit

Vamos criar um *app* para conversar com o livro da Alice utilizando o *chainlit* https://docs.chainlit.io/.

Rode a célula abaixo para criar um arquivo `app.py`:

In [ ]:
%%writefile app.py
"""Alice no País das Maravilhas - RAG
This is a simple RAG (Retrieval-Augmented Generation) application that uses LangChain and
Azure OpenAI to answer questions about the book "Alice in Wonderland".
It uses Chroma as the vector database and SentenceTransformer for embeddings.
It is designed to be run in a Chainlit app.
"""

import os
from typing import List
import chainlit as cl
from chromadb.config import Settings
from dotenv import load_dotenv
from langchain.embeddings.base import Embeddings
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import SentenceTransformer

load_dotenv(override=True)


class SentenceTransformerEmbeddings(Embeddings):
    """Wrapper around SentenceTransformer to use with LangChain."""

    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents."""
        return self.model.encode(texts)

    def embed_query(self, text: str) -> List[float]:
        """Embed a query."""
        return self.model.encode([text])[0]


def singleton(cls):
    """Decorator to make a class a singleton.
    This decorator ensures that a class has only one instance
    and provides a global point of access to it.
    """
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance


@singleton
class MyRAG:
    """Singleton class for the RAG chain."""

    def __init__(self):

        def get_prompt():
            """Get the prompt template."""
            prompt_template = """
Você é um assistente para tarefas de resposta a perguntas. Use as seguintes partes do contexto recuperado para responder à pergunta. Se você não sabe a resposta, basta dizer que não sabe. Use no máximo três frases e mantenha a resposta concisa. Negue qualquer informação que não esteja no contexto recuperado.
Pergunta: {question} 

Contexto: {context} 

Resposta:
"""

            prompt = PromptTemplate(
                template=prompt_template, input_variables=["question", "context"]
            )

            return prompt

        def get_llm():
            """Get the LLM. Use Azure OpenAI."""
            llm = AzureChatOpenAI(
                azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
                api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
                temperature=0,
                max_tokens=int(os.getenv("MAX_TOKENS", "1000")),
                timeout=int(os.getenv("TIMEOUT", "30")),
                max_retries=2,
                model=os.getenv("LLM_MODEL"),
            )
            return llm

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        def get_chroma_settings():
            """Get Chroma settings. Opt out of telemetry."""
            return Settings(anonymized_telemetry=False)

        def get_retriever():
            embedding_model_name = os.getenv("EMBEDDING_MODEL")
            embedding_model = SentenceTransformerEmbeddings(
                model_name=embedding_model_name
            )

            num_vectors = int(os.getenv("NUM_VECTORS"))

            vector_db_dir = os.getenv("VECTOR_DB_DIR")

            retriever = Chroma(
                embedding_function=embedding_model,
                persist_directory=vector_db_dir,
                client_settings=get_chroma_settings(),
            ).as_retriever(num_vectors=num_vectors)

            return retriever

        self.rag_chain = (
            {
                "context": get_retriever() | format_docs,
                "question": RunnablePassthrough(),
            }
            | get_prompt()
            | get_llm()
            | StrOutputParser()
        )

    def invoke(self, pergunta):
        """Invoke the RAG chain with the given question."""
        try:
            return self.rag_chain.invoke(pergunta)
        except Exception as e:
            return f"Erro: {e}"


@cl.on_chat_start
async def on_chat_start():
    """Send a welcome message when the chat starts."""
    await cl.Message(
        content="""Oi!
Sou a assistente virtual da Alice no País das Maravilhas!
Me faça perguntas sobre o livro!"""
    ).send()


@cl.on_message
async def main(message: cl.Message):
    """Handle incoming messages."""
    resp = MyRAG().invoke(message.content)
    await cl.Message(content=resp).send()


Preencha o arquivo `.env` a partir do `.env.example` (faça uma cópia).

Então, no terminal, inicialize a aplicação chainlit com:

```console
chainlit run app.py
```

Acesse, em seu navegador, a URL fornecida, provavelmente http://localhost:8000

Pronto, você criou uma aplicação RAG!

## Referências

- https://en.wikipedia.org/wiki/Vector_database
- https://platform.openai.com/docs/guides/embeddings
- https://platform.openai.com/tokenizer
- https://python.langchain.com/v0.1/docs/use_cases/question_answering/sources/
- https://semaphoreci.com/blog/word-embeddings
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/
- Imagens:
    - https://docs.trychroma.com/img/hrm4.svg
    - https://storage.googleapis.com/gweb-cloudblog-publish/images/image4_fUvNRO7.max-800x800.png
- Livro: http://www.ebooksbrasil.org/eLibris/alicep.html autorizado para uso didático conforme http://www.ebooksbrasil.org/
- https://python.langchain.com/api_reference/openai/llms/langchain_openai.llms.azure.AzureOpenAI.html
- https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.azure.AzureChatOpenAI.html
- https://sbert.net/
- https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
- https://huggingface.co/sentence-transformers
- https://github.com/langchain-ai/langchain/discussions/7818